# Setup

In [1]:
# base
import matplotlib.pyplot as plt
plt.style.use('dark_background')
import seaborn as sns
import pandas as pd
import numpy as np
import datetime

In [2]:
import sys
sys.path.insert(1, '/home/mauricio/code/mcr')
from mcr.util import glimpse, plot_value_counts, plot_value_counts_timeseries, missing_report, plot_missing, plot_unique, plot_duplicates, size
from mcr.spark import roem

In [3]:
from pyspark import SparkContext
# SparkContext.getOrCreate(conf: Optional[pyspark.conf.SparkConf] = None) -> 'SparkContext'
# Amount of memory to use per executor process
# SparkContext.setSystemProperty('spark.driver.cores', '1g') #only in cluster mode
SparkContext.setSystemProperty('spark.driver.memory', '16g')
SparkContext.setSystemProperty('spark.executor.memory', '3g')
SparkContext.setSystemProperty('spark.executor.cores', '16')
sc = SparkContext.getOrCreate()
# sc.setLogLevel('ERROR')  # ALL, DEBUG, ERROR, FATAL, INFO, OFF, TRACE, WARN
# https://intellipaat.com/community/18452/spark-gives-a-stackoverflowerror-when-training-using-als
sc.setCheckpointDir('checkpoint/')
sc.getConf().getAll()

23/05/10 19:41:41 WARN Utils: Your hostname, rig resolves to a loopback address: 127.0.1.1; using 192.168.0.102 instead (on interface enp6s0)
23/05/10 19:41:41 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/10 19:41:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


[('spark.app.id', 'local-1683758502026'),
 ('spark.executor.memory', '3g'),
 ('spark.app.submitTime', '1683758501528'),
 ('spark.executor.id', 'driver'),
 ('spark.driver.memory', '16g'),
 ('spark.driver.host', '192.168.0.102'),
 ('spark.app.name', 'pyspark-shell'),
 ('spark.driver.extraJavaOptions',
  '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UN

In [4]:
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[*]').appName('spark_application').getOrCreate()
print(spark.version)

3.4.0


# Overview of binary, implicit ratings

## Binary ratings
Notice that all ratings are either a 1 or a 0. We must treat binary ratings like these as implicit ratings. If we treated them like explicit ratings and didn't include the 0's, the best performing model would simply predict 1 for everything, and deliver a deceivingly ideal RMSE of 0.

Also, as with our previous Million Songs model, we can't use the RMSE as a model evaluation metric.

Ultimately, when our machine learning process holds out random observations in the test set, we want our model to generate high predictions for those movies that users have actually watched. For this reason, we'll use our ROEM metric again. We'll apply the same concepts we've covered previously on this binary dataset.

The convenience of using the MovieLens dataset is that we can see how our binary model performs against the original, true preference ratings of the original MovieLens dataset. 

In [5]:
# Read data from CSV file
movie_ratings = spark.read.csv('ratings.csv',
                               sep=',',
                               header=True,
                               inferSchema=True).drop('timestamp')

In [6]:
movie_ratings.rdd.getNumPartitions(), movie_ratings.count()

(1, 100004)

In [7]:
users = movie_ratings.select("userId").distinct().orderBy('userId')

In [8]:
users.rdd.getNumPartitions(), users.count()

(1, 671)

In [9]:
movies = movie_ratings.select("movieId").distinct().orderBy('movieId')

In [10]:
movies.rdd.getNumPartitions(), movies.count()

(1, 9066)

In [11]:
cross_join = users.crossJoin(movies)

In [12]:
cross_join.rdd.getNumPartitions(), cross_join.count()

(1, 6083286)

In [13]:
binary_movie_ratings = cross_join\
    .join(movie_ratings, ["userId", "movieId"], "left").distinct().orderBy(['userId', 'movieId'])\
    .fillna(0, subset='rating')

In [14]:
binary_movie_ratings.rdd.getNumPartitions(), binary_movie_ratings.count()

(17, 6083286)

In [15]:
binary_movie_ratings = binary_movie_ratings\
    .withColumn('viewed', (F.col('rating')>0).cast('integer'))\
    .drop('rating')

In [16]:
binary_movie_ratings.rdd.getNumPartitions(), binary_movie_ratings.count()

(17, 6083286)

In [17]:
binary_movie_ratings = binary_movie_ratings.coalesce(1).persist()

In [18]:
binary_movie_ratings.rdd.getNumPartitions(), binary_movie_ratings.count()

(1, 6083286)

## Class imbalance

One word about binary models. While it's perfectly feasible to feed binary data like this into ALS and get meaningful recommendations, the data does have a sort of class imbalance where the vast majority of ratings are 0's with a small percentage of 1's.

**Since implicit ratings models use customized error metrics like ROEM and not RMSE, the class imbalance doesn't really pose a problem like it might in classification problems.**

ALS can still generate meaningful recommendations from this type of data but there are strategies that can be taken with the data to try and improve recommendations. 

In [19]:
print("Sparsity: ", 1 - (movie_ratings.count() / (users.count() * movies.count())))

Sparsity:  0.9835608583913366


## Item weighting
* Item Weighting: Movies with more user views = higher weight

## Item weighting and user weighting
* Item Weighting: Movies with more user views = higher weight
* User Weighting: Users that have seen more movies will have lower weights applied to unseen movies

## Exercises

### Binary model performance

In [20]:
from pyspark.ml.recommendation import ALS

In [21]:
# Empty list to be filled with models
model_list = []
# Complete each of the hyperparameter value lists
ranks = [10, 20, 30, 40]
maxIters = [10, 20, 30, 40]
regParams = [.05, .1, .15]
alphas = [20, 40, 60, 80]
# For loop will automatically create and store ALS models
for r in ranks:
    for mi in maxIters:
        for rp in regParams:
            for a in alphas:
                model_list.append(ALS(userCol= "userId", itemCol= "movieId",
                                      ratingCol= "viewed", rank = r, maxIter = mi, regParam = rp,
                                      alpha = a, coldStartStrategy="drop",nonnegative = True,
                                      implicitPrefs = True))
print(len(model_list))

192


In [22]:
# Split the data into training and test sets
(training, test) = binary_movie_ratings.randomSplit([0.8, 0.2], seed=1)
training = training.persist()
test = test.persist()
print(f'Partitions: Data={binary_movie_ratings.rdd.getNumPartitions()} Train={training.rdd.getNumPartitions()} Test={test.rdd.getNumPartitions()}')
data_count = binary_movie_ratings.count()
training_count = training.count()
test_count = test.count()
print(f'Counts: Data={data_count} Train={training_count} Test={test_count} Difference={data_count-training_count-test_count}')

Partitions: Data=1 Train=1 Test=1


Counts: Data=6083286 Train=4867485 Test=1215801 Difference=0


In [23]:
#Building 5 folds within the training set.
train1, train2, train3, train4, train5 = training.randomSplit([0.2, 0.2, 0.2, 0.2, 0.2], seed=1)
print(f'Partitions: T1={train1.rdd.getNumPartitions()} T2={train2.rdd.getNumPartitions()} T3={train3.rdd.getNumPartitions()} T4={train4.rdd.getNumPartitions()} T5={train5.rdd.getNumPartitions()}')
train1_count = train1.count()
train2_count = train2.count()
train3_count = train3.count()
train4_count = train4.count()
train5_count = train5.count()
print(f'Counts: T1={train1_count} T2={train2_count} T3={train3_count} T4={train4_count} T5={train5_count} Difference={training_count-train1_count-train2_count-train3_count-train4_count-train5_count}')

Partitions: T1=1 T2=1 T3=1 T4=1 T5=1
Counts: T1=974355 T2=972252 T3=975341 T4=973115 T5=972422 Difference=0


In [24]:
fold1 = train2.union(train3).union(train4).union(train5).persist()
fold2 = train3.union(train4).union(train5).union(train1).persist()
fold3 = train4.union(train5).union(train1).union(train2).persist()
fold4 = train5.union(train1).union(train2).union(train3).persist()
fold5 = train1.union(train2).union(train3).union(train4).persist()
foldlist = [(fold1, train1), (fold2, train2), (fold3, train3), (fold4, train4), (fold5, train5)]
print(f'Partitions: F1={fold1.rdd.getNumPartitions()} F2={fold2.rdd.getNumPartitions()} F3={fold3.rdd.getNumPartitions()} F4={fold4.rdd.getNumPartitions()} F5={fold5.rdd.getNumPartitions()}')
fold1_count = fold1.count()
fold2_count = fold2.count()
fold3_count = fold3.count()
fold4_count = fold4.count()
fold5_count = fold5.count()
print(f'Counts: F1={fold1_count} F2={fold2_count} F3={fold3_count} F4={fold4_count} F5={fold5_count}\n')
print('Differences:', end=' ')
print(f'F1={fold1_count-train2_count-train3_count-train4_count-train5_count}', end=' ')
print(f'F2={fold2_count-train3_count-train4_count-train5_count-train1_count}', end=' ')
print(f'F3={fold3_count-train4_count-train5_count-train1_count-train2_count}', end=' ')
print(f'F4={fold4_count-train5_count-train1_count-train2_count-train3_count}', end=' ')
print(f'F5={fold5_count-train1_count-train2_count-train3_count-train4_count}')

Partitions: F1=4 F2=4 F3=4 F4=4 F5=4


Counts: F1=3893130 F2=3895233 F3=3892144 F4=3894370 F5=3895063

Differences: F1=0 F2=0 F3=0 F4=0 F5=0


In [25]:
from time import time
ROEMS = []
for model_num, model in enumerate(model_list):
    print(f'Training model {model_num}')
    fold_num = 1
    roem_sum = 0
    t = time()
    for ft_pair in foldlist:
        # Fits model to fold within training data
        fitted_model = model.fit(ft_pair[0])
        # Generates predictions using fitted_model on respective CV test data
        predictions = fitted_model.transform(ft_pair[1])
        # Generates and prints a ROEM metric CV test data
        r = roem(spark, predictions, 'userId', 'viewed')
        # print(f"\tROEM (fold #{fold_num}): {r}")
        roem_sum += r
        fold_num += 1
    print(f'\tAverage ROEM {roem_sum/(fold_num-1)}')
    # Fits model to all of training data and generates preds for test data
    v_fitted_model = model.fit(training)
    v_predictions = v_fitted_model.transform(test)
    v_ROEM = roem(spark, v_predictions, 'userId', 'viewed')
    # Adds validation ROEM to ROEM list
    ROEMS.append(v_ROEM)
    print (f'\tValidation ROEM: {v_ROEM}')
    print(f"\tElapsed {time()-t:.0f}s")

Training model 0
	Average ROEM 0.13504418790222456


	Validation ROEM: 0.132365951689422
	Elapsed 34s
Training model 1
	Average ROEM 0.13961932221069306
	Validation ROEM: 0.13781299079125872
	Elapsed 30s
Training model 2


	Average ROEM 0.1427257801109573
	Validation ROEM: 0.14193794703142493
	Elapsed 32s
Training model 3
	Average ROEM 0.14527778463950947


	Validation ROEM: 0.1462752625535619
	Elapsed 32s
Training model 4
	Average ROEM 0.13487474275309044


	Validation ROEM: 0.1316352504010507
	Elapsed 31s
Training model 5
	Average ROEM 0.13957516725874347


...

	Validation ROEM: 0.1447523930609863
	Elapsed 117s
Training model 190


	Average ROEM 0.15007043728413372


	Validation ROEM: 0.1479646746276781
	Elapsed 121s
Training model 191


	Average ROEM 0.15248293101484625


	Validation ROEM: 0.15018115568720378
	Elapsed 121s


In [26]:
# Find the index of the smallest ROEM
i = np.argmin(ROEMS)
print(f'Smallest ROEM #{i}: {ROEMS[i]}')

Smallest ROEM #44: 0.12848390693085782


In [27]:
# Extract the best_model
best_model = model_list[i]
# Extract the Rank
print ("Rank: ", best_model.getRank())
# Extract the MaxIter value
print ("MaxIter: ", best_model.getMaxIter())
# Extract the RegParam value
print ("RegParam: ", best_model.getRegParam())
# Extract the Alpha value
print ("Alpha: ", best_model.getAlpha())

Rank:  10
MaxIter:  40
RegParam:  0.15
Alpha:  20.0


In [30]:
fitted_model = best_model.fit(training)
binary_test_predictions = fitted_model.transform(test)
# Look at the test predictions
binary_test_predictions.show(3)
# Evaluate ROEM on test predictions
print(f"ROEM: {roem(spark, binary_test_predictions, 'userId', 'viewed')}")
# Look at user 42's test predictions
binary_test_predictions.filter(F.col("userId") == 42).show(3)

+------+-------+------+-----------+
|userId|movieId|viewed| prediction|
+------+-------+------+-----------+
|   148|      5|     0| 0.42935294|
|   148|      7|     0| 0.45020267|
|   148|     11|     0|  0.4760792|
|   148|     13|     0|0.102432765|
|   148|     15|     0|0.092653215|
|   148|     30|     0| 0.22208145|
|   148|     34|     0| 0.78784674|
|   148|     38|     0| 0.07041129|
|   148|     42|     0| 0.13359056|
|   148|     53|     0|        0.0|
|   148|     54|     0|  0.0121748|
|   148|     58|     1| 0.46741182|
|   148|     60|     0| 0.19647813|
|   148|     63|     0| 0.08127988|
|   148|     74|     0| 0.18537197|
|   148|     81|     0| 0.25807658|
|   148|     93|     0|0.050872304|
|   148|     97|     0| 0.12778923|
|   148|    105|     0|  0.4534478|
|   148|    121|     0|  0.1340127|
+------+-------+------+-----------+
only showing top 20 rows

ROEM: 0.12848390693085782
+------+-------+------+------------+
|userId|movieId|viewed|  prediction|
+------+--

> The model has a pretty low ROEM. Did you notice that the model predicted some high numbers for unseen movies? This indicates that the model is creating recommendations from the movies that users have not seen.

### Recommendations from binary data

In [45]:
# Read data from CSV file
movies = spark.read.csv('movies.csv', sep=',', header=True, inferSchema=True)
original_ratings = movie_ratings.join(movies, on='movieId')

In [46]:
# View user 26's original ratings
print ("User 26 Original Ratings:")
original_ratings.filter(F.col("userId") == 26).show()

User 26 Original Ratings:
+-------+------+------+--------------------+--------------------+
|movieId|userId|rating|               title|              genres|
+-------+------+------+--------------------+--------------------+
|      1|    26|   5.0|    Toy Story (1995)|Adventure|Animati...|
|     32|    26|   4.5|Twelve Monkeys (a...|Mystery|Sci-Fi|Th...|
|     47|    26|   4.5|Seven (a.k.a. Se7...|    Mystery|Thriller|
|     50|    26|   4.5|Usual Suspects, T...|Crime|Mystery|Thr...|
|     63|    26|   0.5|Don't Be a Menace...|        Comedy|Crime|
|     69|    26|   4.5|       Friday (1995)|              Comedy|
|    153|    26|   3.5|Batman Forever (1...|Action|Adventure|...|
|    165|    26|   2.5|Die Hard: With a ...|Action|Crime|Thri...|
|    260|    26|   3.0|Star Wars: Episod...|Action|Adventure|...|
|    296|    26|   3.5| Pulp Fiction (1994)|Comedy|Crime|Dram...|
|    316|    26|   0.5|     Stargate (1994)|Action|Adventure|...|
|    318|    26|   4.0|Shawshank Redempt...|      

In [54]:
binary_recs = fitted_model.transform(binary_movie_ratings).join(movies, on='movieId')
# View user 26's recommendations
print ("User 26 Recommendations:")
binary_recs.filter(F.col("userId") == 26).orderBy('prediction', ascending=False).show()

User 26 Recommendations:
+-------+------+------+----------+--------------------+--------------------+
|movieId|userId|viewed|prediction|               title|              genres|
+-------+------+------+----------+--------------------+--------------------+
|    318|    26|     1| 1.1315075|Shawshank Redempt...|         Crime|Drama|
|    356|    26|     1| 1.1144809| Forrest Gump (1994)|Comedy|Drama|Roma...|
|  58559|    26|     1| 1.1115212|Dark Knight, The ...|Action|Crime|Dram...|
|    296|    26|     1| 1.0668908| Pulp Fiction (1994)|Comedy|Crime|Dram...|
|    593|    26|     1| 1.0519606|Silence of the La...|Crime|Horror|Thri...|
|  46578|    26|     0|  1.033709|Little Miss Sunsh...|Adventure|Comedy|...|
|  60069|    26|     1| 1.0318687|       WALL·E (2008)|Adventure|Animati...|
|   4993|    26|     0| 1.0294702|Lord of the Rings...|   Adventure|Fantasy|
|  72998|    26|     1| 1.0273633|       Avatar (2009)|Action|Adventure|...|
|  63082|    26|     1| 1.0272381|Slumdog Millionai

In [55]:
# View user 99's original ratings
print ("User 99 Original Ratings:")
original_ratings.filter(F.col("userId") == 99).show()

User 99 Original Ratings:
+-------+------+------+--------------------+--------------------+
|movieId|userId|rating|               title|              genres|
+-------+------+------+--------------------+--------------------+
|      1|    99|   4.0|    Toy Story (1995)|Adventure|Animati...|
|      2|    99|   2.0|      Jumanji (1995)|Adventure|Childre...|
|     17|    99|   3.0|Sense and Sensibi...|       Drama|Romance|
|     28|    99|   3.0|   Persuasion (1995)|       Drama|Romance|
|     39|    99|   4.0|     Clueless (1995)|      Comedy|Romance|
|     47|    99|   1.0|Seven (a.k.a. Se7...|    Mystery|Thriller|
|     50|    99|   4.0|Usual Suspects, T...|Crime|Mystery|Thr...|
|    105|    99|   3.0|Bridges of Madiso...|       Drama|Romance|
|    110|    99|   3.0|   Braveheart (1995)|    Action|Drama|War|
|    154|    99|   3.0|Beauty of the Day...|               Drama|
|    160|    99|   2.0|        Congo (1995)|Action|Adventure|...|
|    170|    99|   2.0|      Hackers (1995)|Action

In [56]:
# View user 99's recommendations
print ("User 99 Recommendations:")
binary_recs.filter(F.col("userId") == 99).orderBy('prediction', ascending=False).show()

User 99 Recommendations:
+-------+------+------+----------+--------------------+--------------------+
|movieId|userId|viewed|prediction|               title|              genres|
+-------+------+------+----------+--------------------+--------------------+
|    608|    99|     1| 1.1151934|        Fargo (1996)|Comedy|Crime|Dram...|
|    260|    99|     0| 1.1001455|Star Wars: Episod...|Action|Adventure|...|
|    858|    99|     0| 1.0485737|Godfather, The (1...|         Crime|Drama|
|     36|    99|     0| 1.0440861|Dead Man Walking ...|         Crime|Drama|
|   1073|    99|     0| 1.0357492|Willy Wonka & the...|Children|Comedy|F...|
|   1617|    99|     1|  1.029409|L.A. Confidential...|Crime|Film-Noir|M...|
|    527|    99|     1| 1.0276374|Schindler's List ...|           Drama|War|
|    780|    99|     0| 1.0226269|Independence Day ...|Action|Adventure|...|
|   1225|    99|     0| 1.0222312|      Amadeus (1984)|               Drama|
|   1097|    99|     0| 1.0154196|E.T. the Extra-Te

# Course recap

## Resources

* [McKinsey&Company: "How Retailers Can Keep Up With Consumers"](https://www.mckinsey.com/industries/retail/our-insights/how-retailers-can-keep-up-with-consumers)

    A paper published by McKinsey and Company discussing the power of recommendation engines like ALS based models.

* [ALS Data Preparation: Wide to Long Function](https://github.com/jamenlong/ALS_expected_percent_rank_cv/blob/master/wide_to_long_function.py)

    Code to build the wide_to_long function discussed in the section about preparing data for ALS.

* [Hu, Koren, Volinsky: "Collaborative Filtering for Implicit Feedback Datasets"](http://yifanhu.net/PUB/cf.pdf)

    White paper that provides the academic background for building ALS models using implicit ratings. I highly recommend reading this paper as it provides a lot of context and insight into how these models work and alternative ways to evaluate them.

* [GitHub Repo: Cross Validation With Implicit Ratings in Pyspark](https://github.com/jamenlong/ALS_expected_percent_rank_cv/blob/master/ROEM_cv.py)

    GitHub link for code that manages the cross validation and model evaluation for implicit ratings models using ALS in Pyspark

* [Pan, Zhou, Cao, Liu, Lukose, Scholz, Yang: "One Class Collaborative Filtering"](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.306.4684&rep=rep1&type=pdf)

    Paper that discusses the math and intuition behind the user-based weighting and item-based weighting methodologies for addressing the class imbalance present in binary ratings models.